<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

import nltk

In [5]:
df = pd.read_pickle('../Data/All_Friends_data.plk')

In [11]:
df.head()

,Speaker,Text,Episode,Scene
0,Monica,"Oh, the way you crushed Mike at ping pong was...",1001,"[Scene Barbados, Monica and Chandler's Room. T..."
1,Chandler,"You know, I'd love to, but I'm a little tired.",1001,"[Scene Barbados, Monica and Chandler's Room. T..."
2,Monica,I'll put a pillowcase over my head.,1001,"[Scene Barbados, Monica and Chandler's Room. T..."
3,Chandler,You're on!,1001,"[Scene Barbados, Monica and Chandler's Room. T..."
4,Phoebe,Hey!,1001,"[Scene Barbados, Monica and Chandler's Room. T..."


In [44]:
[i.replace('(', ' ').replace(')', ' ').split(' ') for i in list(df.Speaker.unique())]

[['Monica'],
 ['Chandler'],
 ['Phoebe'],
 ['Ross'],
 ['Charlie'],
 ['Rachel'],
 ['Joey'],
 ['Monica,', 'Chandler', 'and', 'Phoebe'],
 ["Joey's", 'voice,', 'but', 'she', 'sees', 'Ross'],
 ["Joey's", 'voice/Ross'],
 ['Rachel/Ross'],
 ['Both'],
 ['Mike'],
 ['Precious'],
 ['Joey', '', 'to', 'Rachel', ''],
 ['Joey', 'and', 'Rachel'],
 ['Frank', 'Jr.'],
 ['Monica', '', 'reads', 'a', 'form', 'in', 'her', 'lap', ''],
 ['Colleen'],
 ['Bill'],
 ['Owen'],
 ['Frank', 'Jr.', '', 'looks', 'at', 'the', 'triplets', ''],
 ['Chandler', '', 'to', 'Monica', ''],
 ['Ross', '', 'on', 'the', 'verge', 'of', 'tears', ''],
 ['Ross', 'and', 'Chandler'],
 ['Assistant'],
 ['Amanda'],
 ['[scene'],
 ['Rachel', 'and', 'Joey'],
 ['Glenda'],
 ['Ross,', 'Rachel', 'and', 'Chandler'],
 ['Rachel', 'and', 'Phoebe'],
 ['Jack', 'and', 'Judy'],
 ['Jack'],
 ['Judy'],
 ['Chandler', '', 'nearly', 'weeping', ''],
 ['Everybody'],
 ['Ross', '', 'stops', 'recording', ''],
 ['Phoebe', '', 'sees', 'the', 'cake', ''],
 ['Joey', '', 'sta

In [17]:
# List of existing names:
Names = nltk.corpus.names.words()